1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [2]:
import pandas as pd
import numpy as np

In [3]:
pd_df = pd.DataFrame({'language': ['Python', 'Javascript', 'Java', 'Ruby', 'R'],
                        }, 
                     index = [1, 2, 3, 4, 5])

In [4]:
pd_df

,language
1,Python
2,Javascript
3,Java
4,Ruby
5,R


In [5]:
pd_df.shape

(5, 1)

In [6]:
df = spark.createDataFrame(pd_df)


In [7]:
df

DataFrame[language: string]

In [8]:
df.show(5)

+----------+
|  language|
+----------+
|    Python|
|Javascript|
|      Java|
|      Ruby|
|         R|
+----------+



In [9]:
df.describe()

DataFrame[summary: string, language: string]

In [10]:
df.describe().show()


+-------+--------+
|summary|language|
+-------+--------+
|  count|       5|
|   mean|    null|
| stddev|    null|
|    min|    Java|
|    max|    Ruby|
+-------+--------+



2. Load the mpg dataset as a spark dataframe.

**Create 1 column of output that contains a message like the one below:**


- The 1999 audi a4 has a 4 cylinder engine.
- For each vehicle.

- Transform the trans column so that it only contains either manual or auto.

In [11]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [12]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import round, concat, sum, min, max, count, avg, mean

#The 1999 audi a4 has a 4 cylinder engine.

mpg.select(concat(lit("The "), mpg.year, lit(" "), mpg.manufacturer, lit(" "), mpg.model, lit(" has a "), mpg.cyl, lit(" engine")).alias("cylinders")).show()


+--------------------+
|           cylinders|
+--------------------+
|The 1999 audi a4 ...|
|The 1999 audi a4 ...|
|The 2008 audi a4 ...|
|The 2008 audi a4 ...|
|The 1999 audi a4 ...|
|The 1999 audi a4 ...|
|The 2008 audi a4 ...|
|The 1999 audi a4 ...|
|The 1999 audi a4 ...|
|The 2008 audi a4 ...|
|The 2008 audi a4 ...|
|The 1999 audi a4 ...|
|The 1999 audi a4 ...|
|The 2008 audi a4 ...|
|The 2008 audi a4 ...|
|The 1999 audi a6 ...|
|The 2008 audi a6 ...|
|The 2008 audi a6 ...|
|The 2008 chevrole...|
|The 2008 chevrole...|
+--------------------+
only showing top 20 rows



In [13]:
# mpg.select(col1).show(5)

In [14]:
from pyspark.sql.functions import when


In [15]:
mpg.select('manufacturer', mpg.trans, when(mpg.trans.contains("auto"), "auto").otherwise("manual").alias("Trans")).show(5)

+------------+----------+------+
|manufacturer|     trans| Trans|
+------------+----------+------+
|        audi|  auto(l5)|  auto|
|        audi|manual(m5)|manual|
|        audi|manual(m6)|manual|
|        audi|  auto(av)|  auto|
|        audi|  auto(l5)|  auto|
+------------+----------+------+
only showing top 5 rows



3. Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

In [16]:
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [17]:
tot = tips.count()
import pyspark.sql.functions as F


tips.groupBy("smoker") \
  .count() \
  .withColumnRenamed('count', 'cnt_per_group') \
  .withColumn('perc_of_count_total', (F.col('cnt_per_group') / tot) * 100 ) \
  .show()

+------+-------------+-------------------+
|smoker|cnt_per_group|perc_of_count_total|
+------+-------------+-------------------+
|    No|          151| 61.885245901639344|
|   Yes|           93| 38.114754098360656|
+------+-------------+-------------------+



In [18]:

tips.groupBy("smoker") \
  .count().show()

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [19]:

tips.groupBy("smoker") \
  .count().withColumnRenamed('count', 'total').show()

+------+-----+
|smoker|total|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [20]:
F

<module 'pyspark.sql.functions' from '/usr/local/anaconda3/lib/python3.7/site-packages/pyspark/sql/functions.py'>

Use the seattle weather dataset referenced in the lesson to answer the questions below.

Convert the temperatures to farenheight.
Which month has the most rain, on average?
Which year was the windiest?
What is the most frequent type of weather in January?
What is the average high and low temperature on sunny days in July in 2013 and 2014?
What percentage of days were rainy in q3 of 2015?
For each year, find what percentage of days it rained (had non-zero precipitation).

In [21]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



In [23]:
from pyspark.sql.functions import col, expr


In [24]:
weather = weather.withColumn(
    "temp_avg", expr("ROUND(temp_min + temp_max) / 2")
).drop("temp_max", "temp_min")
weather.show(6)

+----------+-------------+----+-------+--------+
|      date|precipitation|wind|weather|temp_avg|
+----------+-------------+----+-------+--------+
|2012-01-01|          0.0| 4.7|drizzle|     9.0|
|2012-01-02|         10.9| 4.5|   rain|     6.5|
|2012-01-03|          0.8| 2.3|   rain|     9.5|
|2012-01-04|         20.3| 4.7|   rain|     9.0|
|2012-01-05|          1.3| 6.1|   rain|     6.0|
|2012-01-06|          2.5| 2.2|   rain|     3.5|
+----------+-------------+----+-------+--------+
only showing top 6 rows



In [25]:
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean


In [30]:
weather.select(max(weather.temp_avg)).show()

+-------------+
|max(temp_avg)|
+-------------+
|         26.5|
+-------------+



In [32]:
from pyspark.sql.functions import month, year, quarter


In [36]:
(
    weather.withColumn("month", month("date"))
    .groupBy("month")
    .agg(max("temp_avg").alias("temp_avg"))
    .sort("temp_avg", ascending=False)
    .show()
)

+-----+--------+
|month|temp_avg|
+-----+--------+
|    8|    26.5|
|    7|    26.0|
|    6|    25.5|
|    9|    25.0|
|    5|    21.5|
|   10|    19.5|
|    4|    18.5|
|    3|    15.5|
|   11|    15.5|
|   12|    14.5|
|    1|    13.0|
|    2|    13.0|
+-----+--------+

